In [12]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer, make_classification
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneOut

from sampler import sample_from_logreg

from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

from cv_svm import SVM_smooth
from kernel_svm import SVM_smooth_kernel

import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [2]:
X, _, y = sample_from_logreg(n=100, p=10, seed=120)
n = X.shape[0]
p = X.shape[1]
y[np.where(y == 0)] = -1

In [3]:
X_scaled = StandardScaler().fit_transform(X)
#X_scaled = X
print(X_scaled.shape)

(100, 10)


In [5]:
# test different lambda values
import time
condition_nos = []
#iacv_iterates = []
#true_iterates = []
sigma = 1e-10
lbd = 1
#length_scales = np.linspace(1e-2, 2, 10)
degrees = np.arange(1, 10)

start = time.time()
for d in degrees:
    #print(f"running experiment for length_scale = {l}")
    print(f"running experiment for degree = {d}")
    clf = SVM_smooth_kernel(sigma=sigma, lbd=lbd, kernel=polynomial_kernel, degree=d)
    clf.fit(X_scaled, y, n_iter=400, eta=0.5/n, approx_cv=True, cv=True,
            save_err_approx=True, save_err_cv=True, log_iacv=False, log_iter=False, save_cond_num=True)
    y_pred = clf.predict(X_scaled)
    score = accuracy_score(y, y_pred)
    
    condition_nos.append(clf.approx_cv_obj.cond_nums)

    #print(f"\tcondition no {np.mean(clf.approx_cv_obj.cond_nums, axis=1)}" )

end = time.time()
print(end - start)

running experiment for degree = 1
running experiment for degree = 2
running experiment for degree = 3
running experiment for degree = 4
running experiment for degree = 5
running experiment for degree = 6


/usr/lib/python3.11/site-packages/numpy/linalg/linalg.py:2582: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
/usr/lib/python3.11/site-packages/numpy/linalg/linalg.py:2583: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
/home/sina/.local/lib/python3.11/site-packages/jax/_src/numpy/array_methods.py:258: RuntimeWarning: overflow encountered in cast
  return binary_op(*args)
/home/sina/Projects/thesis/svm/kernel_svm.py:300: RuntimeWarning: overflow encountered in cast
  - loss_vmap(self.true_cv_obj.iterates, self.gram_, y, self.sigma_)
/home/sina/Projects/thesis/svm/kernel_svm.py:304: RuntimeWarning: overflow encountered in cast
  loss_vmap_fixed_w(self.u_, self.gram_, y, self.sigma_)
/home/sina/Projects/thesis/svm/kernel_svm.py:305: RuntimeWarning: overflow encountered in cast
  - loss_vmap(self.true_cv_obj.iterates, self.gram_, y, self.sigma_)
/home/sina/Projects/thesis/svm/iacv.py:12: RuntimeWarning:

running experiment for degree = 7


/usr/lib/python3.11/site-packages/numpy/linalg/linalg.py:2582: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
/usr/lib/python3.11/site-packages/numpy/linalg/linalg.py:2583: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
/home/sina/.local/lib/python3.11/site-packages/jax/_src/numpy/array_methods.py:258: RuntimeWarning: overflow encountered in cast
  return binary_op(*args)
/home/sina/Projects/thesis/svm/kernel_svm.py:300: RuntimeWarning: overflow encountered in cast
  - loss_vmap(self.true_cv_obj.iterates, self.gram_, y, self.sigma_)
/home/sina/Projects/thesis/svm/kernel_svm.py:304: RuntimeWarning: overflow encountered in cast
  loss_vmap_fixed_w(self.u_, self.gram_, y, self.sigma_)
/home/sina/Projects/thesis/svm/kernel_svm.py:305: RuntimeWarning: overflow encountered in cast
  - loss_vmap(self.true_cv_obj.iterates, self.gram_, y, self.sigma_)
/home/sina/Projects/thesis/svm/iacv.py:12: RuntimeWarning:

LinAlgError: SVD did not converge

In [6]:
mean_cond = np.asarray(condition_nos)
mean_cond = np.mean(np.mean(condition_nos, axis=1), axis=1)

In [13]:
fig, ax = plt.subplots()
ax.plot(degrees[:6], mean_cond)
#ax.set_xscale("log")
ax.set_xlabel("$d$")
ax.set_ylabel("Mean Condition Number")
ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
plt.show()
plt.savefig('poly_d_cond_no.pgf', bbox_inches='tight', pad_inches=0)

/tmp/ipykernel_12022/892099767.py:9: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
